In [ ]:
import numpy as np
from numba import cuda
import time
import os

def create_dummy_particle_file(filename="particles.bin", num_particles=1_000_000):

@cuda.jit
def potential_energy_kernel(masses, positions, potential_energies):
    """
    Numba CUDA 核心，用於計算每個粒子的重力位能。
    每個執行緒計算一個粒子 (i) 的位能貢獻。
    """
    i = cuda.grid(1)
    if i >= positions.shape[0]:
        return

    # G = 6.67430e-11  # 萬有引力常數
    G = 1.0
    
    local_energy = 0.0
    m_i = masses[i]
    pos_i = positions[i]

    # 計算粒子 i 與所有粒子 j (j > i) 之間的位能
    for j in range(i + 1, positions.shape[0]):
        m_j = masses[j]
        pos_j = positions[j]
        
        dx = pos_i[0] - pos_j[0]
        dy = pos_i[1] - pos_j[1]
        dz = pos_i[2] - pos_j[2]
        
        # 避免計算與自身的距離 (雖然迴圈已避免，但這是個好習慣)
        # 並加上一個很小的數 (epsilon) 來避免 r_sq == 0 的情況
        r_sq = dx*dx + dy*dy + dz*dz
        if r_sq < 1e-12: # epsilon^2
            r_sq = 1e-12
        inv_r = 1.0 / cuda.sqrt(r_sq)
        local_energy -= G * m_i * m_j * inv_r

    # 使用原子操作將局部能量加到總能量中
    cuda.atomic.add(potential_energies, 0, local_energy)

def calculate_potential_energy_gpu(filename="particles.bin"):
    """
    使用 Numba 在 GPU 上計算總重力位能。
    """
    # 每個粒子的數據結構：[mass, x, y, z, vx, vy, vz] (7個 float64)
    particle_dtype = np.dtype([
        ('mass', np.float64),
        ('x', np.float64), ('y', np.float64), ('z', np.float64),
        ('vx', np.float64), ('vy', np.float64), ('vz', np.float64)
    ])
    
    # 讀取二進位檔案
    try:
        with open(filename, 'rb') as f:
            binary_data = f.read()
        particles = np.frombuffer(binary_data, dtype=particle_dtype)
    except FileNotFoundError:
        print(f"錯誤: 找不到檔案 '{filename}'。請先生成或提供此檔案。")
        return

    num_particles = len(particles)
    print(f"成功讀取 {num_particles} 個粒子。")

    # 提取質量和位置
    masses = particles['mass']
    positions = np.vstack((particles['x'], particles['y'], particles['z'])).T.copy()

    start_time = time.time()

    # 將數據傳輸到 GPU
    d_masses = cuda.to_device(masses)
    d_positions = cuda.to_device(positions)
    d_potential_energies = cuda.to_device(np.zeros(1, dtype=np.float64))

    # 設定 CUDA 核心的執行配置
    threads_per_block = 256
    blocks_per_grid = (num_particles + (threads_per_block - 1)) // threads_per_block

    # 執行 CUDA 核心
    potential_energy_kernel[blocks_per_grid, threads_per_block](
        d_masses, d_positions, d_potential_energies
    )
    
    # 將結果從 GPU 傳回 CPU
    total_potential_energy = d_potential_energies.copy_to_host()

    end_time = time.time()

    print(f"計算完成。")
    print(f"總重力位能: {total_potential_energy[0]:.6e} J")
    print(f"GPU 計算耗時: {end_time - start_time:.4f} 秒")

if __name__ == "__main__":
    calculate_potential_energy_gpu()